In [18]:
# !pip install pandas
import torch
import torchvision
import torchvision.transforms as transforms
import glob
import os

import torch.nn as nn
import torchvision.datasets as dsets
import pandas as pd
from torchvision.io import read_image
import numpy as np
from PIL import Image

In [19]:
os.getcwd()

'/home/ram/classification-using-MobileNetV2'

In [20]:
# ls

In [21]:
annotations_file='test.csv'
train =pd.read_csv(annotations_file)
train=train.iloc[:,0]
img_dir='images'
for idx ,i in enumerate(train):
    if idx<10:
        img_path = os.path.join(img_dir,i)
        images = read_image(img_path)
        print(images.shape)
    else:
        break

torch.Size([3, 988, 1660])
torch.Size([3, 231, 424])
torch.Size([3, 398, 654])
torch.Size([3, 115, 164])
torch.Size([3, 107, 134])
torch.Size([3, 57, 81])
torch.Size([3, 350, 592])
torch.Size([3, 357, 399])
torch.Size([3, 988, 1660])
torch.Size([3, 231, 424])


In [22]:

class CustomDatasetFromImages():
    def __init__(self,img_dir, csv_path,transforms=None):
        """
        Args:
            csv_path (string): path to csv file
            img_path (string): path to the folder where images are
            transform: pytorch transforms for transforms and tensor conversion
        """
        #image Dir
        self.img_dir = img_dir
        # Read the csv file
        self.df = pd.read_csv(csv_path)
        # First column contains the image paths and image name
        self.image_arr = self.df.iloc[:, 0]
        # forth column is for an label
        self.label = self.df.iloc[:, 3]
        self.transforms = transforms
        
    def __len__(self):
        return len(self.label)


    def __getitem__(self,idx):
        
        img_path = os.path.join(self.img_dir, self.image_arr[idx])
        images = read_image(img_path)
        label = self.label[idx]
        if self.transforms is not None:
            images = self.transforms(images)

        return images , label
            
    def visualize(self):
        print(self.img_dir)
        
        for idx in range(len(self.label)):
            if idx <10:
                # print("images path:",self.image_arr)
                # print('label :',self.label)
                img_path = os.path.join(self.img_dir, self.image_arr[idx])
                images = read_image(img_path)
                label = self.label[idx]

                print(images.shape,label)
                print(type(images))
                # print(images.shape)
            else:
                break

    
    
if __name__ == "__main__":
    # Call dataset
    img_dir ='images'
    annotations_file = 'train.csv'
    customfrom_images = CustomDatasetFromImages(img_dir,annotations_file)
    customfrom_images.visualize()

images
torch.Size([3, 96, 182]) 0
<class 'torch.Tensor'>
torch.Size([3, 235, 325]) 0
<class 'torch.Tensor'>
torch.Size([3, 595, 1176]) 0
<class 'torch.Tensor'>
torch.Size([3, 254, 404]) 0
<class 'torch.Tensor'>
torch.Size([3, 394, 546]) 0
<class 'torch.Tensor'>
torch.Size([3, 785, 561]) 0
<class 'torch.Tensor'>
torch.Size([3, 619, 477]) 0
<class 'torch.Tensor'>
torch.Size([3, 162, 266]) 0
<class 'torch.Tensor'>
torch.Size([3, 276, 410]) 0
<class 'torch.Tensor'>
torch.Size([3, 192, 332]) 0
<class 'torch.Tensor'>


In [23]:
batch_size=5
img_dir ='images'
annotations_file = "train.csv"
input_size = 224



transforms = transforms.Compose([transforms.ToPILImage(),transforms.Resize([int(224), int(224)]),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_dataset = CustomDatasetFromImages(img_dir,annotations_file,transforms=transforms )
img_dir ='images'
annotations_file = "val.csv"

val_dataset = CustomDatasetFromImages(img_dir,annotations_file ,transforms=transforms)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True,
                                            pin_memory=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset ,batch_size=batch_size, shuffle=False, pin_memory=True)





In [24]:
for i,(images,labels) in enumerate(train_loader):
    if i<10:
        print(images.shape ,labels )
    else:
        break

torch.Size([5, 3, 224, 224]) tensor([65, 47, 19, 66, 14])
torch.Size([5, 3, 224, 224]) tensor([37, 15, 80, 77, 26])
torch.Size([5, 3, 224, 224]) tensor([ 9,  7, 40, 22, 80])
torch.Size([5, 3, 224, 224]) tensor([81, 54, 80, 42, 11])
torch.Size([5, 3, 224, 224]) tensor([77, 68, 48, 37, 50])
torch.Size([5, 3, 224, 224]) tensor([55, 75, 56, 38, 28])
torch.Size([5, 3, 224, 224]) tensor([63, 78, 75,  4,  3])
torch.Size([5, 3, 224, 224]) tensor([21, 32, 67, 30, 12])
torch.Size([5, 3, 224, 224]) tensor([ 3, 46, 75, 18, 13])
torch.Size([5, 3, 224, 224]) tensor([59, 23,  3, 72, 15])


In [25]:
# !pip install torchsummary
import torch.nn as nn
from torchsummary import summary

def dwise_conv(ch_in, stride=1):
    return (
        nn.Sequential(
            #depthwise
            nn.Conv2d(ch_in, ch_in, kernel_size=3, padding=1, stride=stride, groups=ch_in, bias=False),
            nn.BatchNorm2d(ch_in),
            nn.ReLU6(inplace=True),
        )
    )

def conv1x1(ch_in, ch_out):
    return (
        nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=1, padding=0, stride=1, bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU6(inplace=True)
        )
    )

def conv3x3(ch_in, ch_out, stride):
    return (
        nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3, padding=1, stride=stride, bias=False),
            nn.BatchNorm2d(ch_out),
            nn.ReLU6(inplace=True)
        )
    )

class InvertedBlock(nn.Module):
    def __init__(self, ch_in, ch_out, expand_ratio, stride):
        super(InvertedBlock, self).__init__()

        self.stride = stride
        assert stride in [1,2]

        hidden_dim = ch_in * expand_ratio

        self.use_res_connect = self.stride==1 and ch_in==ch_out

        layers = []
        if expand_ratio != 1:
            layers.append(conv1x1(ch_in, hidden_dim))
        layers.extend([
            #dw
            dwise_conv(hidden_dim, stride=stride),
            #pw
            conv1x1(hidden_dim, ch_out)
        ])

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.layers(x)
        else:
            return self.layers(x)

class MobileNetV2(nn.Module):
    def __init__(self, ch_in=3, n_classes=1000):
        super(MobileNetV2, self).__init__()

        self.configs=[
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1]
        ]

        self.stem_conv = conv3x3(ch_in, 32, stride=2)

        layers = []
        input_channel = 32
        for t, c, n, s in self.configs:
            for i in range(n):
                stride = s if i == 0 else 1
                layers.append(InvertedBlock(ch_in=input_channel, ch_out=c, expand_ratio=t, stride=stride))
                input_channel = c

        self.layers = nn.Sequential(*layers)

        self.last_conv = conv1x1(input_channel, 1280)

        self.classifier = nn.Sequential(
            nn.Dropout2d(0.2),
            nn.Linear(1280, n_classes)
        )
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.stem_conv(x)
        x = self.layers(x)
        x = self.last_conv(x)
        x = self.avg_pool(x).view(-1, 1280)
        x = self.classifier(x)
        return x


# if __name__=="__main__":
#     # model check
#     model = MobileNetV2(ch_in=3, n_classes=1000)
#     summary(model, (3, 224, 224), device='cpu')

In [26]:
model = MobileNetV2(ch_in=3, n_classes=84)

In [26]:
# removing images with specific size using python
import os
from PIL import Image
folder_images = "dat"
for filenames in os.walk(folder_images):
    img = Image.open(filenames)
    h, w = img.shape
    if(not (h >= 160 or w >= 160)):
        os.remove(img)